# Imports

In [14]:
import numpy as np
import pandas as pd
import sklearn

In [15]:
data_path = "../eugene_weather_data/data.csv"
data = pd.read_csv(data_path)

# Null Values in Data (Pre-Cleaned)


In [16]:
def display_number_of_null(data):
    data_is_null = data.isnull().sum()
    data_is_null = data_is_null.to_frame(name="Amount Null")
    data_is_null["Percent Null"] = ((data_is_null["Amount Null"] / len(data)) * 100).round(2)

    print("\nNumber of data points: ", np.array(data).shape[0], "\n\n")
    print(data_is_null)
    
display_number_of_null(data)
data['Observation_Date'] = pd.to_datetime(data['Observation_Date'])

data['year'] = data['Observation_Date'].dt.year
data['month'] = data['Observation_Date'].dt.month
data['day'] = data['Observation_Date'].dt.day
data = data.drop(columns='Observation_Date')
print(data.describe())


Number of data points:  9144 


                              Amount Null  Percent Null
Station_ID                              0          0.00
Station_Name                            0          0.00
Observation_Date                        0          0.00
Avg_Wind_Speed                          9          0.10
Time_Fastest_Mile                    4799         52.48
Time_Peak_Gust                       4908         53.67
Precipitation                           0          0.00
Percent_Sunshine                     9141         99.97
Snowfall                             6406         70.06
Snow_Depth                           8637         94.46
Avg_Temperature                      2812         30.75
Max_Temperature                         0          0.00
Min_Temperature                         0          0.00
Total_Sunshine                       8027         87.78
Fastest_2Min_Wind_Direction             8          0.09
Fastest_5Sec_Wind_Direction            89          0.97
Snow_Water_Equi

It is clear that the number of missing values is large. To address the issue of missing data in an intellgent manner, I have identified different methods for different features. 

## Avg_Wind_Speed
For Avg_Wind_Speed, which has 0.1% missing data, I am going to attempt to use regression to fill in the values. This feature is continuous and because of the amount of training data we have, in the context of reggression, I think this method is a good fit for filling in missing data. For training data, I chose to use drop features that have missing values, with an excpetion for Fastest_2Min_Wind_Direction and Fastest_2Min_Wind_Speed because they were only missing 0.09% each. For those cases, I dropped the training examples where the Fastest_2Min_Wind_Direction and Fastest_2Min_Wind_Speed were missing. 

In [8]:
def normalize_data_not_date(data, columns_to_scale, columns_to_avoid_scaling):
    scaler = sklearn.preprocessing.StandardScaler() # initalizing a scaler for data normalization
    scaled_data = scaler.fit_transform(data[columns_to_scale])
    scaled_data = pd.DataFrame(scaled_data, columns=columns_to_scale, index=data.index)
    scaled_data = pd.concat([scaled_data, data[columns_to_avoid_scaling]], axis=1)
    return scaled_data

In [9]:

avg_wind_speed_reg_model = sklearn.linear_model.LinearRegression()
rf_model = sklearn.ensemble.RandomForestRegressor(n_estimators=100, random_state=42)

dropped_columns = [
    "Station_ID",
    "Station_Name",
    "Time_Fastest_Mile",
    "Time_Peak_Gust",
    "Percent_Sunshine",
    "Snowfall",
    "Snow_Depth",
    "Avg_Temperature",
    "Total_Sunshine",
    "Fastest_2Min_Wind_Direction",
    "Fastest_5Sec_Wind_Direction",
    "Snow_Water_Equivalent",
    "Fastest_2Min_Wind_Speed",
    "Fastest_5Sec_Wind_Speed",
    "Fog_IceFog_HeavyFog",
    "Heavy_Fog_Mist",
    "Thunderstorms",
    "Ice_Pellets",
    "Hail",
    "Glaze_Rime",
    "Dust_BlowingDust_VolcanicAsh",
    "Smoke_Haze",
    "Blowing_Drifting_Snow",
    "Tornado_Funnel_Cloud",
    "Damaging_Winds",
    "Blowing_Spray",
    "Drizzle",
    "Freezing_Drizzle",
    "Rain",
    "Freezing_Rain",
    "Snow",
    "Snow_IcePellets_OnGround",
    "Ground_Fog",
    "Ice_Fog",
    "Wave_Height_Specific_Period",
    "Significant_Wave_Height",
]



avg_wind_speed_data = data.drop(columns=dropped_columns) 
rows_with_null_targets = avg_wind_speed_data[avg_wind_speed_data['Avg_Wind_Speed'].isnull()] # pulling out our missing values
avg_wind_speed_data_cleaned = avg_wind_speed_data.dropna(subset=['Avg_Wind_Speed'])

training_avg_wind_speed_data, non_train_data = sklearn.model_selection.train_test_split(avg_wind_speed_data_cleaned, test_size=0.3, random_state=42)
validation_avg_wind_speed_data, test_avg_wind_speed_data = sklearn.model_selection.train_test_split(non_train_data, test_size=0.5, random_state=42)

columns_to_scale = ['Precipitation', 'Min_Temperature', "Max_Temperature"]
columns_to_avoid_scaling = ['year', 'month', 'day']

# training data
train_features = training_avg_wind_speed_data.drop(columns='Avg_Wind_Speed')
scaled_train_data = normalize_data_not_date(train_features, columns_to_scale, columns_to_avoid_scaling)
train_targets = training_avg_wind_speed_data['Avg_Wind_Speed']


# Validation data
validation_features = validation_avg_wind_speed_data.drop(columns='Avg_Wind_Speed')
scaled_validation_data = normalize_data_not_date(validation_features, columns_to_scale, columns_to_avoid_scaling)
validation_targets = validation_avg_wind_speed_data['Avg_Wind_Speed']

# Test data
test_features = test_avg_wind_speed_data.drop(columns='Avg_Wind_Speed')
scaled_test_data = normalize_data_not_date(test_features, columns_to_scale, columns_to_avoid_scaling)
test_targets = test_avg_wind_speed_data['Avg_Wind_Speed']

# inputs for prediction
prediction_targets = rows_with_null_targets.drop(columns='Avg_Wind_Speed')


avg_wind_speed_reg_model.fit(scaled_train_data, train_targets)

param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'max_features': ['sqrt', 'log2']
}
#grid_search = sklearn.model_selection.GridSearchCV(estimator=rf_model, param_grid=param_grid, scoring='r2', cv=3, verbose=2)
#grid_search.fit(scaled_train_data, train_targets)
#print(f"Best Parameters: {rf_model.best_params_}")

rf_model.fit(scaled_train_data, train_targets)


validation_predictions_reg = avg_wind_speed_reg_model.predict(scaled_validation_data)
validation_predictions_rf = rf_model.predict(scaled_validation_data)
mse = sklearn.metrics.mean_squared_error(validation_targets, validation_predictions_reg)
r2_reg = sklearn.metrics.r2_score(validation_targets, validation_predictions_reg)
r2_rf = sklearn.metrics.r2_score(validation_targets, validation_predictions_rf)
print(f"R² Score Regression: {r2_reg}")
print(f"R² Score RandomForest: {r2_rf}")
print(f"MSE: {mse}")



R² Score Regression: 0.17589548819773504
R² Score RandomForest: 0.2502913553667515
MSE: 6.758842488616038


# Preset-value Imputation

In this section, we will handling missing values through preset-value imputation. For the below code, I chose to set all null boolean values to 0.

In [10]:

boolean_variables = [ 
    "Fog_IceFog_HeavyFog",
    "Heavy_Fog_Mist",
    "Thunderstorms",
    "Ice_Pellets",
    "Hail",
    "Glaze_Rime",
    "Dust_BlowingDust_VolcanicAsh",
    "Smoke_Haze",
    "Blowing_Drifting_Snow",
    "Tornado_Funnel_Cloud",
    "Damaging_Winds",
    "Blowing_Spray",
    "Drizzle",
    "Freezing_Drizzle",
    "Rain",
    "Freezing_Rain",
    "Snow",
    "Snow_IcePellets_OnGround",
    "Ground_Fog",
    "Ice_Fog",
    "Wave_Height_Specific_Period",
    "Significant_Wave_Height"
]

boolean_filled_data = data
boolean_filled_data[boolean_variables] = boolean_filled_data[boolean_variables].fillna(0)
display_number_of_null(boolean_filled_data)



Number of data points:  9144 


                              Amount Null  Percent Null
Station_ID                              0          0.00
Station_Name                            0          0.00
Avg_Wind_Speed                          9          0.10
Time_Fastest_Mile                    4799         52.48
Time_Peak_Gust                       4908         53.67
Precipitation                           0          0.00
Percent_Sunshine                     9141         99.97
Snowfall                             6406         70.06
Snow_Depth                           8637         94.46
Avg_Temperature                      2812         30.75
Max_Temperature                         0          0.00
Min_Temperature                         0          0.00
Total_Sunshine                       8027         87.78
Fastest_2Min_Wind_Direction             8          0.09
Fastest_5Sec_Wind_Direction            89          0.97
Snow_Water_Equivalent                5126         56.06
Fastest_2Min_Wi

## Dropping Avg_Wind_Speed values

It's important to note that dropping the rows with missing average wind speed results in the removal of all the rows that are missing fastest 2 minutes wind direction and fastest two minute wind speed. Therefore, by dropping rows that are missing the avgerage wind speed data, which contained 9 rows, we also drop the 8 rows missing from fastest 2 minutes wind direction and fastest two minute wind speed. 

In [11]:
data_without_null_avg_wind_speed = data
data_without_null_avg_wind_speed = data_without_null_avg_wind_speed.dropna(subset=["Avg_Wind_Speed"])
display_number_of_null(data_without_null_avg_wind_speed)


Number of data points:  9135 


                              Amount Null  Percent Null
Station_ID                              0          0.00
Station_Name                            0          0.00
Avg_Wind_Speed                          0          0.00
Time_Fastest_Mile                    4790         52.44
Time_Peak_Gust                       4899         53.63
Precipitation                           0          0.00
Percent_Sunshine                     9132         99.97
Snowfall                             6406         70.13
Snow_Depth                           8637         94.55
Avg_Temperature                      2810         30.76
Max_Temperature                         0          0.00
Min_Temperature                         0          0.00
Total_Sunshine                       8018         87.77
Fastest_2Min_Wind_Direction             0          0.00
Fastest_5Sec_Wind_Direction            80          0.88
Snow_Water_Equivalent                5117         56.02
Fastest_2Min_Wi

## Handling Additional Columns

Regarding percent sunshine: this column was dropped becasue it represents a continous value with 99.97% being null. When values are present, they are merely 0. 

Regarding Snowfall and Snow depth, I first computed the snowfall using the formula Snow = Rain * snow_coefficient. I located an article online that provided the coefficients: https://www.omnicalculator.com/other/rain-to-snow#rain-to-snow-ratio. 

Upoon doing this, my percent of null Snowfall, droped from 70.13% to  28.52%. For the remaining number of null values, I chose to simply set them to 0 as the likelyhood of having snow fall when temperatures are greater than 45 degrees F is unlikely. This dropped my snowfall amount to 0%. To tackle snow depth, I opted to compute recent snow fall and then make an assessment on the likely snow depth. Really what I intended here was to find the low hanging fruit. I hoped that most of the null values in snow depth would be set to 0 because there was no recent snow events. What I opted to do was compute the snow depth, by taking a rolling window approach on snow fall. What I did was take the current day, previous, and next days snowfall and add them together for an estimate of snow depth. While this is obviously an estimate, I opted for this route believing that maintaing the information was better than not. In my findings, the majority of the snow depth values were 0 (6760 out of 9135). 

Regarding Avg_Temperature, I opted for the simple approach of taking (max_temp + min_temp) / 2.

Additionally, I decided to drop Snow_Water_Equivalent, as I didn't feel that it would add significant value to our question of ridership and 56.02% of the vlaues were missing. 

Total_Sunshine, Time_Fastest_Mile, and Time_Peak_Gust were other columns that I had a significant number of null values at 87.77%, 52.44%, and 53.63% respectively. Because I do not have a large amounts of meaningful features to train with for estimating these null values, as many of them are sparse boolean features, I don't believe that I can train a model to estimate these values with confidence. Therefore, I opted to drop these rows. 


In [12]:
cleaned_data = data_without_null_avg_wind_speed.drop(columns='Percent_Sunshine')

def rain_to_snow_conversion(data): # computing snowfall in inches
    data.loc[(data['Min_Temperature'] >= 34) & (data['Min_Temperature'] < 45), "Snowfall"] = data['Precipitation'] * 0.1
    data.loc[(data['Min_Temperature'] >= 27) & (data['Min_Temperature'] < 34), "Snowfall"] = data['Precipitation'] * 10
    data.loc[(data['Min_Temperature'] >= 20) & (data['Min_Temperature'] < 27), "Snowfall"] = data['Precipitation'] * 15
    data.loc[(data['Min_Temperature'] >= 15) & (data['Min_Temperature'] < 20), "Snowfall"] = data['Precipitation'] * 20
    data.loc[(data['Min_Temperature'] >= 10) & (data['Min_Temperature'] < 15), "Snowfall"] = data['Precipitation'] * 30
    data.loc[(data['Min_Temperature'] >= 0) & (data['Min_Temperature'] < 10), "Snowfall"] = data['Precipitation'] * 40
    data.loc[(data['Min_Temperature'] >= -20) & (data['Min_Temperature'] < 0), "Snowfall"] = data['Precipitation'] * 50
    data.loc[(data['Min_Temperature'] < -20), "Snowfall"] = data['Precipitation'] * 100
    return data

cleaned_data = rain_to_snow_conversion(cleaned_data)
cleaned_data.loc[cleaned_data["Min_Temperature"] > 32, "Snowfall"] = cleaned_data.loc[cleaned_data["Min_Temperature"] > 32, "Snowfall"].fillna(0)

def compute_snow_depth(df, col='Snowfall', window=5):
    df['Past_Snowfall'] = df[col].rolling(window=window, min_periods=1).sum()
    df["Next_Snowfall"] = df[col][::-1].rolling(window=window, min_periods=1).sum()[::-1]
    df["Snow_Depth"] = df["Past_Snowfall"] + df["Next_Snowfall"]
    df.drop(columns=["Past_Snowfall", "Next_Snowfall"], inplace=True)
    return df 
cleaned_data = compute_snow_depth(cleaned_data, window=1)

cleaned_data['Avg_Temperature'] = (cleaned_data['Min_Temperature'] + cleaned_data['Max_Temperature']) / 2

cleaned_data.drop(columns=['Snow_Water_Equivalent'], inplace=True)
cleaned_data.drop(columns=["Time_Fastest_Mile", 'Time_Peak_Gust', 'Total_Sunshine'], inplace=True)


display_number_of_null(cleaned_data)


Number of data points:  9135 


                              Amount Null  Percent Null
Station_ID                              0          0.00
Station_Name                            0          0.00
Avg_Wind_Speed                          0          0.00
Precipitation                           0          0.00
Snowfall                                0          0.00
Snow_Depth                              0          0.00
Avg_Temperature                         0          0.00
Max_Temperature                         0          0.00
Min_Temperature                         0          0.00
Fastest_2Min_Wind_Direction             0          0.00
Fastest_5Sec_Wind_Direction            80          0.88
Fastest_2Min_Wind_Speed                 0          0.00
Fastest_5Sec_Wind_Speed                80          0.88
Fog_IceFog_HeavyFog                     0          0.00
Heavy_Fog_Mist                          0          0.00
Thunderstorms                           0          0.00
Ice_Pellets    

## Estimating Null Values for Fastest_2Min_Wind_Direction and Fastest_2Min_Wind_Speed

Upon completing the above approach, my dataset was nearly full with the exception of Fastest_2Min_Wind_Direction and Fastest_2Min_Wind_Speed which were both missing 80 datapoints. While dropping the 80 entries is a fine approach, I wanted to attempt to estimate these missing values using a linear regression model so I could retain the 80 entries. 

In [13]:
fastest_reg_model = sklearn.linear_model.LinearRegression()

# pulling out the values
rows_with_null_targets_direction = cleaned_data[cleaned_data['Fastest_2Min_Wind_Direction'].isnull()] # pulling out our missing values
rows_with_null_targets_speed = cleaned_data[cleaned_data['Fastest_2Min_Wind_Speed'].isnull()] # pulling out our missing values

# Dropping values
cleaned_data.dropna(subset=['Fastest_2Min_Wind_Direction'], inplace=True)
cleaned_data.dropna(subset=['Fastest_2Min_Wind_Speed'], inplace=True)

# pulling out test and validation for Fastest_2Min_Wind_Direction
training_direction, non_train_data_direction = sklearn.model_selection.train_test_split(cleaned_data, test_size=0.3, random_state=42)
validation_direction, test_direction = sklearn.model_selection.train_test_split(non_train_data_direction, test_size=0.5, random_state=42)

columns_to_scale = ['Avg_Wind_Speed', 'Snow_Depth', 'Percipitation', ]



# pulling out test and validation for Fastest_2Min_Wind_Direction
training_speed, non_train_data_speed = sklearn.model_selection.train_test_split(cleaned_data, test_size=0.3, random_state=42)
validation_speed, test_direction = sklearn.model_selection.train_test_split(non_train_data_speed, test_size=0.5, random_state=42)

print(cleaned_data)

       Station_ID                      Station_Name  Avg_Wind_Speed  \
0     USW00024221  EUGENE MAHLON SWEET FIELD, OR US            8.28   
1     USW00024221  EUGENE MAHLON SWEET FIELD, OR US            8.28   
2     USW00024221  EUGENE MAHLON SWEET FIELD, OR US            9.84   
3     USW00024221  EUGENE MAHLON SWEET FIELD, OR US           10.07   
4     USW00024221  EUGENE MAHLON SWEET FIELD, OR US            4.25   
...           ...                               ...             ...   
9137  USW00024221  EUGENE MAHLON SWEET FIELD, OR US            5.82   
9138  USW00024221  EUGENE MAHLON SWEET FIELD, OR US            2.68   
9139  USW00024221  EUGENE MAHLON SWEET FIELD, OR US            2.91   
9140  USW00024221  EUGENE MAHLON SWEET FIELD, OR US            1.57   
9141  USW00024221  EUGENE MAHLON SWEET FIELD, OR US            4.92   

      Precipitation  Snowfall  Snow_Depth  Avg_Temperature  Max_Temperature  \
0              0.42     0.042       0.084             42.0          